In [2]:
import pandas as pd
import zipfile
import numpy as np
from os import getcwd
import os

from helper import update_df, get_skeleton_set, extract_year
%load_ext autoreload
%autoreload 2


getcwd()

'/Users/saniyasavla/Desktop/school/sem 2/VIP/ml-vip-vxv-EPFME/codes'

In [26]:
data_dir = f"{getcwd()}/../data/raw/2020q2"
df = pd.read_csv(f"{data_dir}/num.txt", sep="\t")


In [27]:
sub = pd.read_csv(f"{data_dir}/sub.txt", sep="\t")

/var/folders/sy/8yjpdhts2wx6v85js75qxcch0000gn/T/ipykernel_93733/2803878830.py:1: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  sub = pd.read_csv(f"{data_dir}/sub.txt", sep="\t")


In [28]:
sub[(sub['form'] == "10-K") | (sub['form'] == "10-K/A")].head(100)

,adsh,cik,name,sic,countryba,stprba,cityba,zipba,bas1,bas2,...,period,fy,fp,filed,accepted,prevrpt,detail,instance,nciks,aciks
1,0000018498-20-000016,18498,GENESCO INC,5661.0,US,TN,NASHVILLE,37217,GENESCO PK 1415 MURFREESBORO RD,NaN,...,20200131.0,2019.0,FY,20200401,2020-04-01 15:29:00.0,0,1,gcofy202010-kq4_htm.xml,1,NaN
6,0000078239-20-000023,78239,PVH CORP. /DE/,2320.0,US,NY,NEW YORK,10016,200 MADISON AVENUE,NaN,...,20200131.0,2019.0,FY,20200401,2020-04-01 16:57:00.0,0,1,pvh10k2019_htm.xml,1,NaN
26,0000885245-20-000011,885245,BUCKLE INC,5651.0,US,NE,KEARNEY,68847,2407 W 24TH ST,NaN,...,20200131.0,2019.0,FY,20200401,2020-04-01 13:22:00.0,0,1,bke-20200201.xml,1,NaN
32,0000912463-20-000020,912463,GUESS INC,2340.0,US,CA,LOS ANGELES,90021,ATTN: JASON MILLER,1444 SOUTH ALAMEDA STREET,...,20200131.0,2019.0,FY,20200401,2020-04-01 17:04:00.0,0,1,a10k-q4fy20_htm.xml,1,NaN
35,0000930413-20-000944,744187,REX AMERICAN RESOURCES CORP,2860.0,US,OH,DAYTON,45459,7720 PARAGON ROAD,NaN,...,20200131.0,2019.0,FY,20200401,2020-04-01 17:06:00.0,0,1,rex-20200131.xml,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267,0001493152-20-006219,1655971,"POSTADS, INC.",7389.0,US,FL,CORAL SPRINGS,33065,2332 N.W. 87 DR.,NaN,...,20191231.0,2019.0,FY,20200413,2020-04-13 13:52:00.0,0,1,pods-20191231.xml,1,NaN
1270,0001493152-20-006224,1398137,"GENERATION ALPHA, INC.",7380.0,US,CA,CARSON,90746,16926 EAST KEEGAN BLVD,NaN,...,20191231.0,2019.0,FY,20200413,2020-04-13 15:13:00.0,0,1,gnal-20191231.xml,1,NaN
1271,0001493152-20-006229,1563536,E-QURE CORP.,5090.0,US,NY,NEW YORK,10023,20 WEST 64TH STREET,SUITE 39G,...,20191231.0,2019.0,FY,20200413,2020-04-13 15:56:00.0,0,1,equr-20191231.xml,1,NaN
1272,0001493152-20-006232,1430523,"VERUS INTERNATIONAL, INC.",5141.0,US,MD,GAITHERSBURG,20878,9841 WASHINGTONIAN BLVD,STE. #390,...,20191031.0,2019.0,FY,20200413,2020-04-13 16:12:00.0,0,1,vrus-20191031.xml,1,NaN


In [29]:
main_df = df[df['adsh'].isin(set(sub[(sub['form'] == "10-K") | (sub['form'] == "10-K/A")]['adsh'].values))]

In [30]:
main_df2 = main_df[(main_df['uom'] == "USD") & ~np.isnan(main_df['value'])]

In [ ]:
from collections import defaultdict

extract_dir = f"{getcwd()}/../data/raw"
tag_sets = defaultdict(lambda: set())
firm_year_set = set()
for folder in os.listdir(extract_dir):
    if not os.path.isdir(f"{extract_dir}/{folder}"):
        continue
    print(f"Starting {folder}")
    num_df = pd.read_csv(f"{extract_dir}/{folder}/num.txt", sep="\t")
    sub_df = pd.read_csv(f"{extract_dir}/{folder}/sub.txt", sep="\t")
    # filter by 10k
    sub_df = sub_df[(sub_df['form'] == "10-K") | (sub_df['form'] == "10-K/A")]
    num_df = num_df[num_df['adsh'].isin(set(sub_df['adsh'].values))]
    num_df = num_df[(num_df['uom'] == "USD") & ~np.isnan(num_df['value'])] # make sure that value is filled in and uom is USD
    new_df, new_firm_year_set, new_tag_set = get_skeleton_set(num_df, sub_df)
    year = folder.split("q")[0]
    tag_sets[year] = tag_sets[year].union(new_tag_set) # combine for current year
    firm_year_set = firm_year_set.union(new_firm_year_set) # combine for all years/deduplicate

    print(f"{folder} finished")


final_tag_set = None
for year, s, in tag_sets.items():
    if final_tag_set is None:
        final_tag_set = s
    else:
        final_tag_set = final_tag_set.intersection(s)

In [ ]:
import pickle
out_dir = f"{getcwd()}/../data/processed"
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
#dump results
with open(f"{getcwd()}/../data/processed/tag_set.pkl", "wb+") as f:
    pickle.dump(final_tag_set, f)

firm_year_set = {(x[0], int(x[1])) for x in firm_year_set}
with open(f"{getcwd()}/../data/processed/firm_year_set.pkl", "wb+") as f:
    pickle.dump(firm_year_set, f)


In [8]:
# create dataframe
import pickle
firm_year_set = pickle.load(open(f"{getcwd()}/../data/processed/firm_year_set.pkl", "rb"))
final_tag_set = pickle.load(open(f"{getcwd()}/../data/processed/tag_set.pkl", "rb"))
output_df = pd.DataFrame(np.nan, index=list(range(len(firm_year_set))), columns=["Name", "Year"] + list(final_tag_set))

In [9]:
names = [x[0] for x in firm_year_set]
years = [x[1] for x in firm_year_set]
output_df["Name"] = names
output_df["Year"] = years

In [10]:
li = output_df[['Name', 'Year']].values.tolist()
li = [tuple(x) for x in li]
firm_year_index_cache = {x: i for i, x in enumerate(li)}

In [34]:
%%time
from helper import get_10k_df, extract_data
num_df, sub_df = get_10k_df(df, sub)
extract_data(output_df, num_df, sub_df, final_tag_set, firm_year_index_cache)

CPU times: user 5.12 s, sys: 150 ms, total: 5.27 s
Wall time: 5.3 s


In [22]:
deferred = output_df["DeferredCompensationLiabilityCurrentAndNoncurrent"]
deferred[deferred.notnull()]

2086      58800000.0
4163       2000000.0
7867       4447000.0
11102      2870000.0
13238     17652000.0
15941      1395466.0
19052      3756000.0
21147      2200000.0
22263      3793000.0
27048      1552774.0
42981      1439526.0
54208       709663.0
54732      1900000.0
71751      1900000.0
71794      6100000.0
88374       755000.0
88642      5400000.0
89062      1042000.0
91379      1903000.0
99468       685000.0
101545    55700000.0
102516     2365000.0
Name: DeferredCompensationLiabilityCurrentAndNoncurrent, dtype: float64

In [ ]:
def get_tag_val(df: pd.DataFrame, tag: str):
    name = df['Name']
    year = df['Year']
    code = sub[sub['name'] == name]
    adsh = code['adsh'].values
    if len(adsh) == 0:
        return np.nan
    adsh = adsh[0]
    row = main_df.loc[(main_df['adsh'] == adsh) & (main_df['tag'] == tag)]
    value = row['value'].values
    if (len(value) == 0):
        return np.nan
    print(value)

test_df = output_df.head(100)

In [ ]:
output_df.to_csv(f"{getcwd()}/../data/processed/skeleton.csv", index=False)

In [3]:
# Packages
import pandas as pd
import datetime as dtt

path_data = f"{getcwd()}/../data/raw/"

# Import CRSP: 
#   PERMNO is security identifier for CRSP, SHRCD is security type, 
#   SICCD is SIC industry categorization. NCUSIP is historical security identifier.
#   COMNAM is company name, PRIMEXCH is primary exchange, CUSIP is current security identifier
#   PRC is price, VOL is volume, SHROUT is shares outstanding, vwretd and ewretd are value and equally weighted market return averages.
#
# you will need date and NCUSIP to identify mainly these companies. May use Companyname and or Ticker with XBRL if there is no/partial CUSIP identifier
df_crsp = pd.read_csv(path_data+'crsp_monthly.csv')
df_crsp['date'] = pd.to_datetime(df_crsp['date'])

# Import IBES
df_ibes = pd.read_csv(path_data+'ibes_annual.csv')
df_ibes['date'] = pd.to_datetime(df_ibes['ANNDATS'])


# YOU SHOULD MERGE IT ON XBRL data, based on the process described in Table 1
# but here is an example how to merge them together:

# Select needed columns
df_crsp_m = df_crsp[['date','NCUSIP','COMNAM','PRC','SHROUT','vwretd','ewretd']]
df_crsp_m.shape
df_ibes_m = df_ibes[['CUSIP','CNAME','date','VALUE']]
df_ibes_m.shape

# Merge: inner (both has to be in data, note you have to modify it to be in XBRL, and it is not the same stuff necessarily)
df = pd.merge(df_ibes_m, df_crsp_m, how='inner', left_on=['date', 'CUSIP'], right_on=['date','NCUSIP'])

# All obs from CRPS and IBES
df.shape

# Drop duplicates
df = df.drop_duplicates(subset=['date','NCUSIP'])
df.shape

# Available stock price from CRSP (non NA)
df_total = df.dropna(subset=['PRC'])
df_total.shape

##
# NOTES: a) you need to carefully investigate and remove the duplicates form CRSP and IBES and XBRL individually! Check the sample sizes! E.g. here you will end up with way less observations

FileNotFoundError: [Errno 2] No such file or directory: '/Users/saniyasavla/Desktop/school/sem 2/VIP/ml-vip-vxv-EPFME/codes/../data/raw/crsp_monthly.csv'

In [12]:
df_total = df_total.rename(columns={'COMNAM':'Name'})
mask = output_df['Name'].isin(df_total['Name'])
merged_df = output_df[mask]
merged_df = merged_df.sort_values(['Name', 'Year'])

In [13]:
merged_df["VALUE"]

KeyError: 'VALUE'

In [14]:
df_total['year'] = pd.to_datetime(df_total['date']).dt.year

In [15]:
test = df_total[(df_total['Name'] == '1ST CONSTITUTION BANCORP') & (df_total['year'] == 2019)]
test['VALUE']

137851    1.56
Name: VALUE, dtype: float64

In [16]:
test

,CUSIP,CNAME,date,VALUE,NCUSIP,Name,PRC,SHROUT,vwretd,ewretd,year
137851,31986N10,1ST CONSTITUTION,2019-01-31,1.56,31986N10,1ST CONSTITUTION BANCORP,19.42,8606.0,0.088294,0.12026,2019
